# Task Scheduling Project
## Run in Google Colab

This notebook will automatically:
1. Install .NET SDK
2. Clone the repository from GitHub
3. Restore and build the project
4. Run all experiments
5. Display results

**Instructions:**
- Click "Runtime" → "Run all" (or press Ctrl+F9)
- Wait for all cells to complete
- Results will be displayed at the bottom


In [ ]:
# Step 1: Install system dependencies and .NET SDK
print("=" * 70)
print("Step 1: Installing .NET SDK")
print("=" * 70)

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libicu-dev libunwind8 libssl-dev curl

# Download and install .NET SDK 6.0 LTS (most stable)
!wget -q https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!bash dotnet-install.sh --channel 6.0 --install-dir /root/.dotnet --verbose

# Set environment variables
import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Verify installation
print("\nVerifying .NET installation...")
result = !export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && dotnet --version 2>&1
if result and len(result) > 0:
    print(f"✅ .NET version: {result[0]}")
else:
    print("❌ .NET installation failed. Trying alternative method...")
    !bash -c 'export PATH=$PATH:/root/.dotnet && dotnet --version'


In [ ]:
# Step 2: Clone repository from GitHub
print("=" * 70)
print("Step 2: Cloning Repository from GitHub")
print("=" * 70)

import os
import shutil

# Remove existing directory if it exists
if os.path.exists('Task-Scheduling'):
    print("Removing existing Task-Scheduling directory...")
    shutil.rmtree('Task-Scheduling')

# Clone repository
print("Cloning repository...")
!git clone https://github.com/MohamadJehad/Task-Scheduling.git

# Change to project directory
os.chdir('Task-Scheduling')
print(f"\n✅ Repository cloned successfully!")
print(f"📁 Current directory: {os.getcwd()}")


In [ ]:
# Step 3: Restore NuGet packages
print("=" * 70)
print("Step 3: Restoring NuGet Packages")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Restore packages
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Restore failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Packages restored successfully!")


In [ ]:
# Step 4: Build the project
print("=" * 70)
print("Step 4: Building Project")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Build project
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet build --no-restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Build failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Project built successfully!")


In [ ]:
# Step 5: Run the project
print("=" * 70)
print("Step 5: Running Task Scheduling Experiments")
print("=" * 70)
print("\nThis may take a few minutes...\n")

import os
import subprocess
import sys

os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Determine the correct project directory path
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Check if we're already in Task-Scheduling or need to navigate to it
if os.path.basename(current_dir) == 'Task-Scheduling':
    project_dir = current_dir
elif os.path.exists(os.path.join(current_dir, 'Task-Scheduling')):
    project_dir = os.path.join(current_dir, 'Task-Scheduling')
else:
    # Try to find it in parent directories
    parent_dir = os.path.dirname(current_dir)
    if os.path.exists(os.path.join(parent_dir, 'Task-Scheduling')):
        project_dir = os.path.join(parent_dir, 'Task-Scheduling')
    else:
        # Default: assume it's in current directory
        project_dir = os.path.join(current_dir, 'Task-Scheduling')

print(f"Project directory: {project_dir}")

if not os.path.exists(project_dir):
    print(f"❌ Error: Project directory not found at {project_dir}")
    print("Available directories:")
    for item in os.listdir('.'):
        if os.path.isdir(item):
            print(f"  - {item}")
else:
    # Change to project directory
    os.chdir(project_dir)
    print(f"✅ Changed to: {os.getcwd()}\n")
    
    # Run the project
    try:
        # Use subprocess with proper environment
        env = os.environ.copy()
        env['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + env.get('PATH', '')
        env['DOTNET_ROOT'] = '/root/.dotnet'
        
        # Run dotnet with the correct working directory
        process = subprocess.Popen(
            ['/root/.dotnet/dotnet', 'run', '--no-build'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True,
            env=env,
            cwd=project_dir
        )
        
        # Stream output in real-time
        for line in process.stdout:
            print(line, end='')
            sys.stdout.flush()
        
        process.wait()
        
        if process.returncode == 0:
            print("\n\n✅ Experiments completed successfully!")
        else:
            print(f"\n\n❌ Process exited with code {process.returncode}")
            
    except Exception as e:
        print(f"\n\n❌ Error running project: {str(e)}")
        print("\nTrying alternative method with bash...")
        !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && export DOTNET_ROOT=/root/.dotnet && cd Task-Scheduling && dotnet run --no-build'


In [ ]:
# Step 6: Display all generated reports
print("=" * 70)
print("Step 6: Generated Reports")
print("=" * 70)

import os
from IPython.display import display, Markdown

# Find the reports directory - check current directory and Task-Scheduling subdirectory
current_dir = os.getcwd()
reports_dir = None

# Check if we're in Task-Scheduling directory
if os.path.basename(current_dir) == 'Task-Scheduling':
    reports_dir = os.path.join(current_dir, 'reports')
elif os.path.exists(os.path.join(current_dir, 'Task-Scheduling', 'reports')):
    reports_dir = os.path.join(current_dir, 'Task-Scheduling', 'reports')
elif os.path.exists(os.path.join(current_dir, 'reports')):
    reports_dir = os.path.join(current_dir, 'reports')
else:
    # Try to find Task-Scheduling directory
    for root, dirs, files in os.walk('.'):
        if 'Task-Scheduling' in root and 'reports' in dirs:
            reports_dir = os.path.join(root, 'reports')
            break

if reports_dir and os.path.exists(reports_dir):
    files = [f for f in os.listdir(reports_dir) if f.endswith('.txt')]
    files.sort()
    
    if files:
        print(f"\n📊 Found {len(files)} report file(s):\n")
        
        # Find Master Summary report
        master_summary = [f for f in files if 'MasterSummary' in f]
        
        if master_summary:
            print(f"\n{'='*70}")
            print(f"📄 MASTER SUMMARY REPORT: {master_summary[0]}")
            print(f"{'='*70}\n")
            
            with open(os.path.join(reports_dir, master_summary[0]), 'r', encoding='utf-8') as f:
                content = f.read()
                # Display as code block for better formatting
                display(Markdown(f"```\n{content}\n```"))
            
            print(f"\n{'='*70}\n")
        
        # List all other reports
        other_reports = [f for f in files if 'MasterSummary' not in f]
        if other_reports:
            print(f"\n📋 Individual Instance Reports ({len(other_reports)} files):")
            for file in other_reports:
                file_path = os.path.join(reports_dir, file)
                file_size = os.path.getsize(file_path)
                print(f"  • {file} ({file_size:,} bytes)")
    else:
        print("\n⚠️  No report files found in reports directory.")
else:
    print(f"\n⚠️  Reports directory not found.")
    print(f"Current directory: {current_dir}")
    print("Searching for reports directory...")
    # List available directories
    for root, dirs, files in os.walk('.'):
        if 'reports' in dirs:
            print(f"  Found reports at: {os.path.join(root, 'reports')}")

print(f"\n{'='*70}")
print("✅ All done! Reports are saved in the 'reports' folder.")
print(f"{'='*70}")
